In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import qp

In [ ]:
f = tables_io.read('/Users/echarles/pz/data/train/dc2_run2p2i_dr6_train_dered.hdf5')
t = tables_io.read('/Users/echarles/pz/data/test/dc2_run2p2i_dr6_test_dered.hdf5')
x = qp.read('/Users/echarles/pz/projects/dc2_run2p2i/data/gold_baseline/output_estimate_knn.hdf5')

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
mags_i = f[f"mag_i_cModel_obj_dered"]
features = np.vstack([mags_i] + [f[f"mag_{band}_cModel_obj_dered"] - mags_i for band in 'ugrzy']).T
mags = np.vstack([f[f"mag_{band}_cModel_obj_dered"] for band in 'ugrizy']).T
targets = f['redshift_true']

In [ ]:
reg = HistGradientBoostingRegressor(max_iter=5000)

In [ ]:
reg.fit(features, targets)

In [ ]:
test_mags_i = t[f"mag_i_cModel_obj_dered"]
test_features = np.vstack([test_mags_i] + [t[f"mag_{band}_cModel_obj_dered"] - test_mags_i for band in 'ugrzy']).T
test_mags = np.vstack([t[f"mag_{band}_cModel_obj_dered"] for band in 'ugrizy']).T
test_targets = t['redshift_true']

In [ ]:
test = reg.predict(test_features)

In [ ]:
_ = plt.hist2d(test_targets, test, bins=np.linspace(0, 3., 301))

In [ ]:
_ = plt.hist2d(test_targets, np.squeeze(x.ancil['zmode']), bins=np.linspace(0, 3., 301), norm="log")

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=6)

In [ ]:
pca.fit(features)

In [ ]:
pca.singular_values_

In [ ]:
pca.transform(features)

In [ ]:
reg2 = HistGradientBoostingRegressor(max_iter=5000)

In [ ]:
reg2.fit(pca.transform(features), targets)

In [ ]:
test2 = reg2.predict(pca.transform(test_features))

In [ ]:
_ = plt.hist2d(test_targets, test2, bins=np.linspace(0, 3., 301))

In [ ]:
from sklearn.cluster import OPTICS

In [ ]:
clustering = OPTICS()

In [ ]:
#clustering.fit(features)

In [ ]:
_ = plt.hist2d(targets, pca.transform(features)[:,3], bins=(50,50), norm='log')